In [12]:
! pip install -qU llama-index llama-index-llms-openai llama-index-readers-file octoai llama-index-llms-octoai llama-index-embeddings-octoai llama-index-llms-openai-like


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.octoai import OctoAI
from llama_index.embeddings.octoai import OctoAIEmbedding

In [14]:
import os

In [7]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [8]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [9]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [10]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [13]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai_like import OpenAILike

/Users/yujian/Documents/workspace/ai_agents_cookbooks/p312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [16]:
llm = OpenAILike(
    model="meta-llama-3.1-70b-instruct",
    api_base="https://text.octoai.run/v1",
    api_key=os.environ["OCTOAI_API_TOKEN"],
    context_window=40000,
    is_function_calling_model=True,
    is_chat_model=True,
)

In [17]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [18]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

> Running step 84955300-597c-4ecf-8c91-76b9527cb85d. Step input: What was Lyft's revenue growth in 2021?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': 'Lyft revenue growth 2021'}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
> Running step e73d5bd7-9fd5-4e66-8a4e-4acb169980c0. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lyft's revenue increased by 36% in 2021 compared to the prior year.
Lyft's revenue increased by 36% in 2021 compared to the prior year.
